In [ ]:
%pip install geopandas

In [ ]:
import geopandas as gpd
import folium

# Chemin vers le fichier shapefile
chemin_shapefile = "Alpes_WGS84/Alpes.shp"

# Charger le shapefile avec GeoPandas
gdf = gpd.read_file(chemin_shapefile)

# Vérifier les données
print(gdf.head())  # Affiche les premières lignes pour vérifier les colonnes disponibles
print(gdf.crs)     # Vérifie le système de coordonnées

# Vérifiez si une colonne contient les noms des massifs (par exemple, 'nom_massif')
nom_colonne_massif = 'nom'  # Remplacez par le nom réel de la colonne

# Filtrer les massifs que vous souhaitez afficher
massifs_a_afficher = unique_values_list  # Remplacez par les noms réels des massifs
print(massifs_a_afficher)
gdf_filtre = gdf[gdf['nom'].isin(massifs_a_afficher)]  # 'nom_massif' est l'attribut contenant les noms des massifs
print(gdf_filtre)

# Centrer la carte sur les Alpes du Nord
centre_lat, centre_lon = 45.8, 6.5  # Latitude et longitude approximatives
carte = folium.Map(location=[centre_lat, centre_lon], zoom_start=9)

# Ajouter les massifs en tant que couches distinctes
for _, row in gdf_filtre.iterrows():
    # Obtenir le nom du massif
    nom_massif = row[nom_colonne_massif]

    # Ajouter un GeoJson pour chaque massif, avec son propre Layer
    folium.GeoJson(
        row['geometry'],  # Géométrie du massif
        name=nom_massif,  # Le nom de la couche dans le contrôle de couches
        style_function=lambda x: {
            "fillColor": "blue",  # Couleur de remplissage
            "color": "black",     # Couleur des contours
            "weight": 1,          # Épaisseur des contours
            "fillOpacity": 0.5    # Transparence
        },
        tooltip=folium.Tooltip(f"Massif : {nom_massif}")  # Affiche le nom en survolant
    ).add_to(carte)

# Ajouter un contrôle de couches
folium.LayerControl().add_to(carte)

# Afficher la carte
carte.save("carte_massifs_couches.html")



In [ ]:
import geopandas as gpd
import folium
from folium.plugins import TimeSliderChoropleth
import pandas as pd
import json

# Charger le shapefile
chemin_shapefile = "Alpes_WGS84/Alpes.shp"
gdf = gpd.read_file(chemin_shapefile)

# Charger le DataFrame contenant les données d'épaisseur de neige
# Exemple fictif de structure du DataFrame df :
# df = pd.DataFrame({
#     'Libelle': ['Massif1', 'Massif2', ...],
#     'Annee': [2020, 2020, ...],
#     'Altitude': [1200, 1500, ...],
#     'Epaisseur': [50, 75, ...]
# })
# Assurez-vous que 'Libelle' correspond à 'nom' dans gdf.

# Vérifiez la correspondance des colonnes
df['Libelle'] = df['Libelle'].str.strip()  # Éviter les problèmes d'espaces
gdf['nom'] = gdf['nom'].str.strip()

# Joindre les données du DataFrame au GeoDataFrame
gdf = gdf.merge(df, left_on='nom', right_on='Libelle', how='inner')

# Normaliser les épaisseurs pour la visualisation (entre 0 et 1)
# gdf['Epaisseur_norm'] = gdf['SNDAV'] / gdf['SNDAV'].max()

# Préparer les données pour TimeSliderChoropleth
styledict = {}
for _, row in gdf.iterrows():
    massif = row['nom']
    if massif not in styledict:
        styledict[massif] = {}
    for annee in sorted(df['Annee'].unique()):
        filtered_row = gdf[(gdf['nom'] == massif) & (gdf['Annee'] == annee)]
        if not filtered_row.empty:
            value = filtered_row['SNDAV'].iloc[0]  # Récupérer la valeur normalisée
        else:
            value = 0  # Valeur par défaut si aucune donnée
        styledict[massif][str(annee)] = {
            'color': 'black',
            'opacity': 0.5,
            'fillColor': 'YlGnBu',
            'fillOpacity': value
        }

# Créer une carte Folium
centre_lat, centre_lon = 45.8, 6.5
carte = folium.Map(location=[centre_lat, centre_lon], zoom_start=9)

# Ajouter le TimeSliderChoropleth
time_slider = TimeSliderChoropleth(
    data=styledict,
    styledict=styledict
)
time_slider.add_to(carte)

# Ajouter une légende
folium.LayerControl().add_to(carte)

# Sauvegarder la carte
carte.save("carte_epaisseur_neige.html")


gdf